In [2]:
######################################################################
###############   IMPORTING THE NECESSARY PACKAGES  ##################
######################################################################

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
######################################################################
####################   DEVICE CONFIGURATION  #########################
######################################################################

device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
######################################################################
#########################  HYPERPARAMETERS ###########################
######################################################################

batch_size = 100
num_classes = 10
num_epochs = 5
learning_rate = 0.001

In [5]:
######################################################################
###############   IMPORTING THE NECESSARY PACKAGES  ##################
######################################################################

train_dataset = torchvision.datasets.MNIST(root = 'data/', train = True, transform = transforms.ToTensor(), download = True)
test_dataset = torchvision.datasets.MNIST(root = 'data/', train = False, transform = transforms.ToTensor())

trainLoader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
testLoader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [7]:
######################################################################
########################   DEFINING THE MODEL   ######################
######################################################################

class ConvNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0),-1)
        out = self.fc(out)
        return(out)
    
model = ConvNet(num_classes = 10).to(device)

In [10]:
######################################################################
################   DEFINING THE LOSS AND CRITERION   #################
######################################################################

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [14]:
######################################################################
######################   TRAINING THE MODEL   ########################
######################################################################

total_step = len(trainLoader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainLoader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward Propagation
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #if (i+1)%300:
            
        if (i+1)%100 == 0:
            print("epoch: [{}/{}], step: [{}/{}], Loss: {:.4f} \n".format(epoch+1,
            num_epochs, i+1, total_step, loss.item()))
            
            

epoch: [0/5], step: [100/600], Loss: 0.1189 

epoch: [0/5], step: [200/600], Loss: 0.1792 

epoch: [0/5], step: [300/600], Loss: 0.0815 

epoch: [0/5], step: [400/600], Loss: 0.0341 

epoch: [0/5], step: [500/600], Loss: 0.0318 

epoch: [0/5], step: [600/600], Loss: 0.0193 

epoch: [1/5], step: [100/600], Loss: 0.0437 

epoch: [1/5], step: [200/600], Loss: 0.0623 

epoch: [1/5], step: [300/600], Loss: 0.0722 

epoch: [1/5], step: [400/600], Loss: 0.0222 

epoch: [1/5], step: [500/600], Loss: 0.0518 

epoch: [1/5], step: [600/600], Loss: 0.0142 

epoch: [2/5], step: [100/600], Loss: 0.0060 

epoch: [2/5], step: [200/600], Loss: 0.0345 

epoch: [2/5], step: [300/600], Loss: 0.0061 

epoch: [2/5], step: [400/600], Loss: 0.0519 

epoch: [2/5], step: [500/600], Loss: 0.0181 

epoch: [2/5], step: [600/600], Loss: 0.0049 

epoch: [3/5], step: [100/600], Loss: 0.0397 

epoch: [3/5], step: [200/600], Loss: 0.0324 

epoch: [3/5], step: [300/600], Loss: 0.0328 

epoch: [3/5], step: [400/600], Los

In [17]:
######################################################################
##########################   TESTING THE MODEL    ####################
######################################################################

# This is for using batchnormalization learned parameter while training
# for testing data
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testLoader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.77 %
